# Data Manipulation Challenge

## Table of Content

1) Introduction <br>

2) Import Libraries <br>

3) Gather Data <br>

4) Remove Columns and Unpivot Data <br>

5) Get Daily Total <br>

6) Merge <br>

7) Export 

## 1. Introduction

The goal of this challenge is to unpivot the data and merge the files. The data is formated with the dates as the column and with cumulative totals. We want the dates by rows with the daily total as a separate column. 

Source: USAFacts US Coronavirus Database via Google BigQuery
Last Update: Sept. 22 2021

## 2. Import Libraries

In [1]:
import pandas as pd
import numpy as np 

## 3. Gather Data

In [2]:
cases = pd.read_csv(r'C:\Users\Chunna\Documents\Data_Analyst_Portfolio\Covid19_USAFacts\confirmed_cases.csv')
deaths = pd.read_csv(r'C:\Users\Chunna\Documents\Data_Analyst_Portfolio\Covid19_USAFacts\deaths.csv')

## 4. Remove Columns and Unpivot Data

In [3]:
# Confirmed Cases
cases_drop = cases.drop(columns=['county_fips_code','state_fips_code'])
cases_unpivot = cases_drop.melt(id_vars=['county_name','state'], var_name='date', value_name='rolling_cases')
cases_unpivot['date'] = cases_unpivot.date.str[1:].replace('_', '-',regex=True)
cases_unpivot['date'] = pd.to_datetime(cases_unpivot['date'])
cases_unpivot = cases_unpivot.sort_values(['state','county_name'])
cases_unpivot.head()

,county_name,state,date,rolling_cases
921,Aleutians East Borough,AK,2020-01-22,0
1897,Aleutians East Borough,AK,2020-01-23,0
2873,Aleutians East Borough,AK,2020-01-24,0
3849,Aleutians East Borough,AK,2020-01-25,0
4825,Aleutians East Borough,AK,2020-01-26,0


In [4]:
# Deaths
deaths_drop = deaths.drop(columns=['county_fips_code','state_fips_code'])
deaths_unpivot = deaths_drop.melt(id_vars=['county_name','state'], var_name='date', value_name='rolling_deaths')
deaths_unpivot['date'] = deaths_unpivot.date.str[1:].replace('_', '-',regex=True)
deaths_unpivot['date'] = pd.to_datetime(deaths_unpivot['date'])
deaths_unpivot = deaths_unpivot.sort_values(['state','county_name'])
deaths_unpivot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 652239 entries, 615 to 651836
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   county_name     652239 non-null  object        
 1   state           652239 non-null  object        
 2   date            652239 non-null  datetime64[ns]
 3   rolling_deaths  652239 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 24.9+ MB


## 5. Get Daily Total

In [5]:
# Confirmed Cases
county_match_d = deaths_unpivot['county_name'] == deaths_unpivot['county_name'].shift(1) 
state_match_d = deaths_unpivot['state'] == deaths_unpivot['state'].shift(1)
date_match_d = deaths_unpivot['date'] == deaths_unpivot['date'].shift(1) + pd.DateOffset(1)
rolling_deaths_1_d = deaths_unpivot['rolling_deaths'].shift(1) == 0
rolling_deaths_2_d = deaths_unpivot['rolling_deaths'] > deaths_unpivot['rolling_deaths'].shift(1) 
rolling_deaths_3_d = deaths_unpivot['rolling_deaths'] == deaths_unpivot['rolling_deaths'].shift(1) 
true_1_d = deaths_unpivot['rolling_deaths'] - deaths_unpivot['rolling_deaths'].shift(1)


deaths_unpivot['daily_deaths'] = np.where(
                                    (county_match_d &
                                    state_match_d &
                                    date_match_d) &
                                    (rolling_deaths_1_d | rolling_deaths_2_d | rolling_deaths_3_d),
                                    true_1_d,
                                    0)
deaths_unpivot

,county_name,state,date,rolling_deaths,daily_deaths
615,Matanuska-Susitna Borough,AK,2020-01-22,0,0.0
1686,Matanuska-Susitna Borough,AK,2020-01-23,0,0.0
2757,Matanuska-Susitna Borough,AK,2020-01-24,0,0.0
3828,Matanuska-Susitna Borough,AK,2020-01-25,0,0.0
4899,Matanuska-Susitna Borough,AK,2020-01-26,0,0.0
...,...,...,...,...,...
647552,Natrona County,WY,2021-09-17,149,0.0
648623,Natrona County,WY,2021-09-18,149,0.0
649694,Natrona County,WY,2021-09-19,149,0.0
650765,Natrona County,WY,2021-09-20,149,0.0


In [6]:
# Deaths
county_match_c = cases_unpivot['county_name'] == cases_unpivot['county_name'].shift(1) 
state_match_c = cases_unpivot['state'] == cases_unpivot['state'].shift(1)
date_match_c = cases_unpivot['date'] == cases_unpivot['date'].shift(1) + pd.DateOffset(1)
rolling_cases_1_c = cases_unpivot['rolling_cases'].shift(1) == 0
rolling_cases_2_c = cases_unpivot['rolling_cases'] > cases_unpivot['rolling_cases'].shift(1) 
rolling_cases_3_c = cases_unpivot['rolling_cases'] == cases_unpivot['rolling_cases'].shift(1) 
true_1_c = cases_unpivot['rolling_cases'] - cases_unpivot['rolling_cases'].shift(1)

cases_unpivot['daily_confirmed_cases'] = np.where(
                                    (county_match_c & 
                                    state_match_c &  
                                    date_match_c) & 
                                    (rolling_cases_1_c | rolling_cases_2_c | rolling_cases_3_c),
                                    true_1_c,
                                    0)
cases_unpivot

,county_name,state,date,rolling_cases,daily_confirmed_cases
921,Aleutians East Borough,AK,2020-01-22,0,0.0
1897,Aleutians East Borough,AK,2020-01-23,0,0.0
2873,Aleutians East Borough,AK,2020-01-24,0,0.0
3849,Aleutians East Borough,AK,2020-01-25,0,0.0
4825,Aleutians East Borough,AK,2020-01-26,0,0.0
...,...,...,...,...,...
590357,Teton County,WY,2021-09-17,4743,17.0
591333,Teton County,WY,2021-09-18,4743,0.0
592309,Teton County,WY,2021-09-19,4743,0.0
593285,Teton County,WY,2021-09-20,4743,0.0


## 6. Merge

In [7]:
df = pd.merge(cases_unpivot,deaths_unpivot)
df

,county_name,state,date,rolling_cases,daily_confirmed_cases,rolling_deaths,daily_deaths
0,Municipality of Anchorage,AK,2020-01-22,0,0.0,0,0.0
1,Municipality of Anchorage,AK,2020-01-23,0,0.0,0,0.0
2,Municipality of Anchorage,AK,2020-01-24,0,0.0,0,0.0
3,Municipality of Anchorage,AK,2020-01-25,0,0.0,0,0.0
4,Municipality of Anchorage,AK,2020-01-26,0,0.0,0,0.0
...,...,...,...,...,...,...,...
390973,Natrona County,WY,2021-09-17,11444,35.0,149,0.0
390974,Natrona County,WY,2021-09-18,11444,0.0,149,0.0
390975,Natrona County,WY,2021-09-19,11444,0.0,149,0.0
390976,Natrona County,WY,2021-09-20,11444,0.0,149,0.0


## 7. Export

In [11]:
df.to_csv('covid19_data.csv', index=False)